In [2]:

import numpy as np
import pandas as pd
from validphys.api import API

In [5]:
fitname = "250210_mtop_variation_test"

In [7]:
fit = API.fit(fit=fitname)

In [20]:
# We have to know the name of the alphas point prescription (alphas_pp) to
# extract the theoryids. We have to know alphas_pp_id to identify the .csv file
# corresponding to the alphas covmat used in the fit
pps = fit.as_input()["theorycovmatconfig"]["point_prescriptions"]
mtop_pp_id, mtop_pp = [[j,i] for j,i in enumerate(pps) if "mtop" in i][0]

common_dict = dict(
    dataset_inputs={"from_": "fit"},
    fit=fit.name,
    fits=[fit.name],
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
)

theoryids_dict = ({
        "point_prescription": mtop_pp,
        "theoryid": {"from_": "theory"},
        "theory": {"from_": "fit"},
        "theorycovmatconfig": {"from_": "fit"},
    } | common_dict)
theoryids = API.theoryids(**theoryids_dict)
theory_plus = theoryids[1].id
theory_mid = theoryids[0].id
theory_min = theoryids[2].id

thcov_input_pdf = fit.as_input()["theorycovmatconfig"]["pdf"]

# Inputs for central theory (used to construct the mtop covmat)
inps_central = dict(theoryid=theory_mid, pdf=thcov_input_pdf, **common_dict)

# Inputs for plus theory (used to construct the mtop covmat)
inps_plus = dict(theoryid=theory_plus, pdf=thcov_input_pdf, **common_dict)

# Inputs for minus theory prediction (used to construct the mtop covmat)
inps_minus = dict(theoryid=theory_min, pdf=thcov_input_pdf, **common_dict)

# inputs for the computation of the prediction of the fit with cov=C+S, where S
# is computed using the inps_central, inps_plus, and inps_minus dictionaries
inps_central_fit = dict(theoryid=theory_mid, pdf={"from_": "fit"}, **common_dict)

In [23]:
if fit.as_input()["resample_negative_pseudodata"] != False:
    print("The TCM assumes Gaussianity of the pseudodata, to ensure this set")
    print("resample_negative_pseudodata: False")
    print("in the n3fit runcard!")

KeyError: 'resample_negative_pseudodata'

In [24]:
prior_theorypreds_central = API.group_result_central_table_no_table(**inps_central)["theory_central"]
prior_theorypreds_plus = API.group_result_central_table_no_table(**inps_plus)["theory_central"]
prior_theorypreds_minus = API.group_result_central_table_no_table(**inps_minus)["theory_central"]

LHAPDF 6.5.4 loading /Users/jaco/miniconda3/envs/nnpdf_dev/share/LHAPDF/240701-02-rs-nnpdf40-baseline/240701-02-rs-nnpdf40-baseline_0000.dat
240701-02-rs-nnpdf40-baseline PDF set, member #0, version 1


In [26]:
# Get the values of mtop...
mtop_plus = API.theory_info_table(theory_db_id=theory_plus).loc["mt"].iloc[0]
mtop_central = API.theory_info_table(theory_db_id=theory_mid).loc["mt"].iloc[0]
mtop_min = API.theory_info_table(theory_db_id=theory_min).loc["mt"].iloc[0]

# ... and make sure the alphas shift in both directions is symmetric
delta_mtop_plus = mtop_plus - mtop_central
delta_mtop_min = mtop_central - mtop_min
if abs(delta_mtop_min - delta_mtop_plus) > 1e-6:
    raise ValueError("mtop shifts in both directions is not symmetric")
else:
    mtop_step_size = delta_mtop_min

In [29]:
# At some point we scaled the covmat to account for higher order derivatives or
# to test depencence of the prior. It is not used in the final result
covmat_scaling_factor = 1 #fit.as_input().get("theorycovmatconfig",{}).get("rescale_alphas_covmat",1.0)

beta_tilde = np.sqrt(covmat_scaling_factor) * (mtop_step_size / np.sqrt(2)) * np.array([1, -1])
S_tilde = beta_tilde @ beta_tilde

In [30]:
delta_plus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_plus - prior_theorypreds_central
)
delta_minus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_minus - prior_theorypreds_central
)

beta = [delta_plus, delta_minus]
S_hat = beta_tilde @ beta

S = np.outer(delta_plus, delta_plus) + np.outer(delta_minus, delta_minus)
S = pd.DataFrame(S, index=delta_minus.index, columns=delta_minus.index)

In [31]:
stored_top_covmat = pd.read_csv(
    fit.path / f"tables/datacuts_theory_theorycovmatconfig_point_prescriptions{mtop_pp_id}_theory_covmat_custom_per_prescription.csv",
    index_col=[0, 1, 2],
    header=[0, 1, 2],
    sep="\t|,",
    encoding="utf-8",
    engine="python",
)

In [33]:
stored_mtop_covmat = pd.read_csv(
    fit.path / f"tables/datacuts_theory_theorycovmatconfig_point_prescriptions{mtop_pp_id}_theory_covmat_custom_per_prescription.csv",
    index_col=[0, 1, 2],
    header=[0, 1, 2],
    sep="\t|,",
    encoding="utf-8",
    engine="python",
).fillna(0)
storedcovmat_index = pd.MultiIndex.from_tuples(
    [(aa, bb, np.int64(cc)) for aa, bb, cc in stored_mtop_covmat.index],
    names=["group", "dataset", "id"],
)  # make sure theoryID is an integer, same as in S
stored_mtop_covmat = pd.DataFrame(
    stored_mtop_covmat.values, index=storedcovmat_index, columns=storedcovmat_index
)
stored_mtop_covmat = stored_mtop_covmat.reindex(S.index).T.reindex(S.index)

if not np.allclose(S, stored_mtop_covmat):
    print("Reconstructed theory covmat, S, is not the same as the stored covmat!")

In [34]:
theorypreds_fit = API.group_result_table_no_table(**inps_central_fit).iloc[:, 2:]

LHAPDF 6.5.4 loading all 101 PDFs in set 250210_mtop_variation_test
250210_mtop_variation_test, version 1; 101 PDF members


In [35]:
exp_covmat = API.groups_covmat(
    use_t0=True,
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
    theoryid= {"from_": "theory"},
    theory={"from_": "fit"},
    **common_dict
)

In [36]:
total_th_covmat = pd.read_csv(
    fit.path / f"tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv",
    index_col=[0, 1, 2],
    header=[0, 1, 2],
    sep="\t|,",
    encoding="utf-8",
    engine="python",
).fillna(0)
storedcovmat_index = pd.MultiIndex.from_tuples(
    [(aa, bb, np.int64(cc)) for aa, bb, cc in total_th_covmat.index],
    names=["group", "dataset", "id"],
)  # make sure theoryID is an integer, same as in S
total_th_covmat = pd.DataFrame(
    total_th_covmat.values, index=storedcovmat_index, columns=storedcovmat_index
)
total_th_covmat = total_th_covmat.reindex(S.index).T.reindex(S.index)

In [37]:
# Note that mean_prediction is different from the prediction of the mean PDF (i.e. replica0)
mean_prediction = theorypreds_fit.mean(axis=1)

X = np.zeros_like(exp_covmat.values)
for i in range(theorypreds_fit.shape[1]):
    X += np.outer(
        (theorypreds_fit.iloc[:, i] - mean_prediction),
        (theorypreds_fit.iloc[:, i] - mean_prediction),
    )
X *= 1 / theorypreds_fit.shape[1]

In [38]:
# In the computation we use <D>_rep and not the central value of the data D_exp, though if
# resample_negative_pseudodata: false
# is set in the n3fit runcard, D_exp and <D>_rep should be the same as N_rep -> inf.
pseudodata = API.read_pdf_pseudodata(**common_dict)
dat_reps = pd.concat(
    [i.pseudodata.reindex(prior_theorypreds_central.index) for i in pseudodata], axis=1
)

In [40]:
invcov = np.linalg.inv(exp_covmat + total_th_covmat)

# delta_T_tilde is Eq. 3.37 in https://arxiv.org/pdf/2105.05114
delta_T_tilde = -S_hat @ invcov @ (mean_prediction - dat_reps.mean(axis=1))

# P_tilde is Eq. 3.38.
#
# Note that not all terms of the equation in the paper are here, in particular
# X_tile and X_hat vanish. This is because they measure the covariance of
# T_tilde over PDF replicas, but for us T_tilde is alphas. The prediciton of
# alphas does not depend on the PDF, and as such T_tilde^(r) == T_tilde^(0)
P_tilde = S_hat.T @ invcov @ X @ invcov @ S_hat + S_tilde - S_hat.T @ invcov @ S_hat

pred = mtop_central + delta_T_tilde
unc = np.sqrt(P_tilde)
print(rf"Prediction for $\m_{{top}}$: {pred:.4f} ± {unc:.4f}")

Prediction for $\m_{top}$: 173.3099 ± 0.5624


In [ ]:
# next: include alphas, MHOU